# Damian Assistant

This notebook trains a Keras multiclass classifier to predict an intent from a given natural language message.
The intents are either general (greeting, goodbye, thanks, noanswer, options) or professional (skills, projects, hobbies, education, experience).

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import random

Using TensorFlow backend.


In [2]:
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi there", "How are you", "Is anyone there?", "Hello", "Good day"],
            "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],
            "context": [""]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
            "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
            "context": [""]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
            "responses": ["Happy to help!", "Any time!", "My pleasure"],
            "context": [""]
        },
        {
            "tag": "noanswer",
            "patterns": [],
            "responses": ["Sorry, can't understand you", "Please give me more info", "Not sure I understand"],
            "context": [""]
        },
        {
            "tag": "options",
            "patterns": ["How you could help me?", "What you can do?", "What help you provide?", "How you can be helpful?"],
            "responses": ["I can tell you about my (Damian's) education, work experience, personal projects, skills and interests", "My primary function is to provide professional information about myself (Damian). Try asking me about technical skills or work experience."],
            "context": [""]
        },
        {
            "tag": "education",
            "patterns": ["What university did you go to?", "Do you have a degree?", "Do you have a BSc in Computer Science?", "Where did you go to uni?"],
            "responses": ["I have a BSc Computer Science degree (First Class Honours) from the University of Hertforshire. My dissertation titled 'An Evaluation of Sentiment Analysis Algorithms'"],
            "context": [""]
        },
        {
            "tag": "skills",
            "patterns": ["Tell me about your technical skills", "What programming languages do you know?", "Are you familiar with Javascript?", "Do you know Python?", "I'm looking for a Java developer."],
            "responses": ["I'm a Full Stack developer, proficient in Javascript (React.js, Node.js), Java and Python. I have experience with both SQL and NoSQL databases and am familiar with CI/CD technologies including Kubernetes, Jenkins and GitHub. I am also familiar with Machine Learning frameworks and libraries including TensorFlow/Keras and scikit-learn."],
            "context": [""]
        },
        {
            "tag": "projects",
            "patterns": ["Where can I see your work?", "Have you written any blogs?", "Tell me about some personal projects?", "Do you have a GitHub account?"],
            "responses": ["I have a number of personal projects listed on my website https://damiancummins.github.io including links to blog posts, github repositories and information about events I've attended."],
            "context": [""]
        },
        {
            "tag": "experience",
            "patterns": ["Tell me about your work experience", "Where have you worked?", "What experience do you have?", "Tell me about your time at IBM"],
            "responses": ["I've worked at IBM since I graduated in 2015 (I also worked at IBM as an intern during my degree). My role as a Software Developer involved in building enterprise grade, cloud-native, Software-as-a-Service (SaaS) solutions in the Data and AI space at IBM, in particular ETL and AI Operations offerings. Most recently, I've been responsible for React.js based UIs, backend Node.js REST microservices and a Python runtime compute engine, with both SQL and NoSQL databases (DB2, PostgreSQL, Cloudant)."],
            "context": [""]
        },
        {
            "tag": "hobbies",
            "patterns": ["What are your hobbies?", "What are your hobbies outside of work?", "What do you do for fun?", "What do you do in your down-time?"],
            "responses": ["I play Five-a-side Football pretty regularly, I go camping and hiking fairly often, and I try to make it out to France/Switzerland over the Ski season. I've also been attending French classes for the past year."],
            "context": [""]
        }
   ]
}

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
stemmer = LancasterStemmer()

words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenise each word
        w = nltk.word_tokenize(pattern)
        # add to the words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), 'documents', documents)
# classes = intents
print(len(classes), 'classes', classes)
# words = all words, vocabulary
print(len(words), 'unique stemmed words', words)

40 documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'university', 'did', 'you', 'go', 'to', '?'], 'education'), (['Do', 'you', 'have', 'a', 'degree', '?'], 'education'), (['Do', 'you', 'have', 'a', 'BSc', 'in', 'Computer', 'Science', '?'], 'education'), (['Where', 'did', 'yo

### Training
Convert the words into bags of words

In [6]:
# create our training data
training = []
# create empty output array
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialise bag of words
    bag = []
    # list of tokenised words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X = patterns, Y = intents
train_x = list(training[:,0])
train_y = list(training[:,1])

### Create model
Create Keras multiclass classification model with 3 layers using Softmax activation.

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains
# number of neurons equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [9]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
40/40 [==============================] - 1s 24ms/step - loss: 2.2789 - acc: 0.1500
Epoch 2/200
40/40 [==============================] - 0s 3ms/step - loss: 2.2213 - acc: 0.1250
Epoch 3/200
40/40 [==============================] - 0s 3ms/step - loss: 2.1819 - acc: 0.0750
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 2.1146 - acc: 0.2750
Epoch 5/200
40/40 [==============================] - 0s 3ms/step - loss: 2.0603 - acc: 0.2750
Epoch 6/200
40/40 [==============================] - 0s 3ms/step - loss: 1.9929 - acc: 0.3500
Epoch 7/200
40/40 [==============================] - 0s 3ms/step - loss: 1.9249 - acc: 0.4750
Epoch 8/200
40/40 [==============================] - 0s 3ms/step - loss: 1.8300 - acc: 0.5500
Epoch 9/200
40/40 [==============================] - 0s 3ms/step - loss: 1.5900 - acc: 0.6000
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 1.8402 - acc: 0.4500
Epoch 11/2

40/40 [==============================] - 0s 3ms/step - loss: 0.0442 - acc: 1.0000
Epoch 170/200
40/40 [==============================] - 0s 3ms/step - loss: 0.0496 - acc: 1.0000
Epoch 171/200
40/40 [==============================] - 0s 2ms/step - loss: 0.0109 - acc: 1.0000
Epoch 172/200
40/40 [==============================] - 0s 2ms/step - loss: 0.0906 - acc: 0.9500
Epoch 173/200
40/40 [==============================] - 0s 3ms/step - loss: 0.0567 - acc: 0.9750
Epoch 174/200
40/40 [==============================] - 0s 2ms/step - loss: 0.0528 - acc: 0.9750
Epoch 175/200
40/40 [==============================] - 0s 2ms/step - loss: 0.0206 - acc: 1.0000
Epoch 176/200
40/40 [==============================] - 0s 3ms/step - loss: 0.0025 - acc: 1.0000
Epoch 177/200
40/40 [==============================] - 0s 3ms/step - loss: 0.0171 - acc: 1.0000
Epoch 178/200
40/40 [==============================] - 0s 2ms/step - loss: 0.0066 - acc: 1.0000
Epoch 179/200
40/40 [==============================] -

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

In [11]:
p = bow("Tell me about your work experience", words)
print(p)
print(classes)

found in bag: tel
found in bag: me
found in bag: about
found in bag: yo
found in bag: work
found in bag: expery
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 0]
['education', 'experience', 'goodbye', 'greeting', 'hobbies', 'options', 'projects', 'skills', 'thanks']


In [12]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print([input_data.values.tolist()])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r > ERROR_THRESHOLD]
    #sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [13]:
classify_local("What hobbies do you have?")

found in bag: what
found in bag: hobby
found in bag: do
found in bag: you
found in bag: hav
[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]]


[('hobbies', '0.9186848')]

In [14]:
model.save('damian_assistant_model.h5')

In [15]:
#Generate a download link for model
from IPython.display import HTML
import base64
import io

def create_download_link_for_kerasmodel( title = "Damian Assistant model", filename = "damian_assistant_model.h5"):  
    
    with open(filename,'rb') as file:
        kerasmodel = file.read()
    b64 = base64.b64encode(kerasmodel)
    payload = b64.decode()

    html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link_for_kerasmodel()